# Feature engineering

<mark>TODO</mark> : à intégrer dans le ZIP auto en tant que Nom_Prénom_1_notebook_exploratoire_mmaaaa

Découverte des données à l'aide de l'analyse exploratoire.

Suppression ou correction des données aberrantes (outliers de niveau 0).

Identification et contrôle d'impact des outliers hors normes non aberrants (outliers de niveau 1).

Identification et assertion des relations (bivariées) constantes et linéaires (outliers de niveau 2).

# Chargements

## Libs

In [1]:
from pepper_commons import *
from pepper_monovar import *
from pepper_bivar import *


OFF COMMONS IS LOADED!

_test_data loaded

struct.json loaded


## Données

Source : https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy

In [2]:
data = get_data()
display(data)
_data = data.copy()      # _data is the working version and data the original version

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,NaN,Compliant,NaN,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,...,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,...,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,...,5.252517e+05,1.792159e+06,39737.390630,3.973739e+06,False,NaN,Compliant,NaN,223.54,16.99
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Center,8319 8th Ave S,Seattle,WA,NaN,7883603155,...,1.022480e+05,3.488702e+05,3706.010010,3.706010e+05,False,NaN,Compliant,NaN,22.11,1.57


# Nettoyages

## Retrait des colonnes vides et constantes

Retrait de la colonne vide `Comments` et de colonnes constantes `DataYear` = `2016`, `City` = `'Seattle'`, `State` = `'WA'`.

In [3]:
print_subtitle('Removal of constants (invariant vars)')
print(bold('DataYear'), 'constant :', data.DataYear.unique())   # 2016, année des relevés
print(bold('City'), 'constant :', data.City.unique())           # Seattle évidemment
print(bold('State'), 'constant :', data.State.unique())         # WA (Washington) dont Seattle est la + gde ville
print(bold('Comments'), 'constant :', data.Comments.unique())   # No comment
_data = _data.drop(columns=['DataYear', 'City', 'State', 'Comments'])
print("⇒ ['DataYear', 'City', 'State', 'Comments'] dropped")


Removal of constants (invariant vars)
DataYear constant : [2016]
City constant : ['Seattle']
State constant : ['WA']
Comments constant : [nan]
⇒ ['DataYear', 'City', 'State', 'Comments'] dropped


## Retrait des outliers 'officiels'

Nous disposons d'une colonne `Outlier` pour identifier les valeurs aberrantes (1 % des cas).

In [6]:
from pepper_production import drop_class
_data, outliers = drop_class(_data, _data.Outlier.notna(), 'outliers')
display(outliers)

TypeError: eval_bindex() got multiple values for argument 'op'

## Conformité (`ComplianceStatus`)

La variable `ComplianceStatus` indique si une propriété satisfait aux exigences en matière d'analyse comparative de l'énergie pour l'année de déclaration en cours. En clair : les individus non compliants sont des données inexploitables, dont les outliers, les données par défaut, les données partielles.

In [ ]:
from pepper_production import drop_class
compliance = _data.ComplianceStatus
print(bold('Modalities'), ':', list(compliance.unique()))
print(compliance.value_counts())
_data, c = drop_class(_data, _data.ComplianceStatus != 'Compliant', 'not compliant')

Modalities : ['Compliant', 'Error - Correct Default Data', 'Missing Data', 'Non-Compliant']
Compliant                       3211
Error - Correct Default Data     111
Missing Data                      15
Non-Compliant                      7
Name: ComplianceStatus, dtype: int64

Removal of not compliant


,OSEBuildingID,BuildingType,PrimaryPropertyType,PropertyName,Address,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,...,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,ComplianceStatus,TotalGHGEmissions,GHGEmissionsIntensity
22,28,SPS-District K-12,K-12 School,Graham Hill Elementary,5149 S. Graham St.,98118.0,1102000138,2,SOUTHEAST,47.54576,...,1713623.5,0.0,274605.9063,936955.0,6025.229004,602523.0,True,Error - Correct Default Data,38.53,0.69
28,34,SPS-District K-12,K-12 School,Meany Building,301/300 21st Ave. E.,98112.0,0688000090,3,Central,47.62266,...,0.0,0.0,0.0000,0.0,0.000000,0.0,False,Missing Data,0.00,0.00
30,36,SPS-District K-12,K-12 School,Jane Addams Middle,11051 34th Ave. N.E.,98125.0,0752000170,5,NORTH,47.70994,...,7800349.5,0.0,570809.1875,1947601.0,48315.718750,4831572.0,True,Error - Correct Default Data,270.18,1.68
31,37,SPS-District K-12,K-12 School,John Hay Elementary,201 Garfield St.,98109.0,0809000005,7,MAGNOLIA / QUEEN ANNE,47.63290,...,0.0,0.0,438691.6875,1496816.0,0.000000,0.0,False,Missing Data,10.43,0.20
38,51,SPS-District K-12,K-12 School,Eckstein Middle,3003 N.E. 75th St.,98115.0,0425049022,4,NORTHEAST,47.68252,...,7329817.5,0.0,468840.5000,1599684.0,47994.433590,4799443.0,True,Error - Correct Default Data,266.05,1.52


⇒ 133 not compliant + ComplianceStatus column dropped
